# SageMaker Edge Manager Example

1. [Introduction](#Introduction)
2. [Demo Setup](#Demo-Setup)
    1. [Launch EC2 Instance](#Launch-EC2-Instance)
3. [Compile Model using SageMaker Neo](#Compile-Model-using-SageMaker-Neo)
    1. [Load pretrained model](#Load-pretrained-model)
6. [Deploy Model using Sagemaker Edge Manager](#Deploy-Model-using-Sagemaker-Edge-Manager)
    1. [Package Model](#Package-Model)
    2. [Create AWS IoT thing](#Create-AWS-IoT-thing)
    3. [Create Device Fleet](#Create-Device-Fleet)
    4. [Create and register client certificate with AWS IoT](#Create-and-register-client-certificate-with-AWS-IoT)
7. [Inference on Edge](#Inference-on-Edge)
    1. [Setup Sagemaker Edge Manager Agent](#Setup-Sagemaker-Edge-Manager-Agent) 
    2. [Load Model](#Load-Model)
    3. [List Models](#List-Models)
    4. [Run Predict](#Run-Predict)
    5. [Capture Data](#Capture-Data)
    6. [Unload Model](#Unload-Model)
8. [Clean Up](#Clean-Up)
9. [Appendix](#Appendix)
    1. [(Optional)Install CloudWatch Agent](#(Optional)Install-CloudWatch-Agent )

## Introduction

SageMaker Edge Manager is a service from Amazon SageMaker that lets you:

+ prepares custom models for edge device hardware
+ includes a runtime for running machine learning inference efficiently on edge devices
+ enables devices to send samples of data from each model securely to SageMaker for relabeling and retraining.

There are two main components to this service:
+ SageMaker Edge Manager in the Cloud 
+ SageMaker Edge Agent on the Edge device

This notebook demonstrates the end-to-end workflow for getting a running SageMaker Edge Agent on the edge device. This will involve the following steps:

+ Compile the model using SageMaker Neo
+ Package the compiled model with SageMaker Edge Manager
+ Deploy with SageMaker Edge Manager Agent
+ Run inference with the model
+ Capture the model's input and output data to S3

**Note**:
Typically, the SageMaker Edge Agent is run on an edge device. For the sake of this notebook, we will run the Agent on an EC2 instance. We show how to package the compiled model and then load it to the Agent on the edge Device to make predictions with. Finally, we show how to capture model's input and output to S3 via the Agent.

This notebook is intented only for notebook instances. When you run this notebook, choose the kernel: `conda_tensorflow_p36`

**Please note**: There are pricing implications to the use of this notebook. Please refer to [Edge Manager](https://aws.amazon.com/sagemaker/edge-manager/pricing) for more information.

## Demo Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and send command with Systems Manager.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
print(role)

Locate the above printed sagemaker role from [IAM console](https://console.aws.amazon.com/iam), find and attach the following policies to role:

- AmazonEC2FullAccess 
- AmazonEC2RoleforSSM 
- AmazonSSMManagedInstanceCore 
- AmazonSSMFullAccess 
- AWSIoTFullAccess 

You can find more information about how to attach policies to role [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console).

**If you try this example with a real device, only attach AWSIoTFullAccess to create certificates on AWS IoT.**

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "DEMO-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)

Finally we upload the test image to S3 bucket. This image will be used in inference later.

In [ ]:
darknet_img_path = sess.upload_data("darknet.bmp", bucket, iot_folder)
keras_img_path = sess.upload_data("keras.bmp", bucket, iot_folder)

### Launch EC2 Instance

As mentioned earlier, this EC2 instance is used in place of an edge device for running the agent software.

In [ ]:
ec2_client = boto3.client("ec2", region_name=region)

Generate a key pair for the EC2 instance, and save the key pem file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance instead.

In [ ]:
import datetime

ec2_key_name = "edge-manager-key-" + str(datetime.datetime.now())
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=ec2_key_name,
)

key_pair = str(ec2_key_pair["KeyMaterial"])
key_pair_file = open("ec2-key-pair.pem", "w")
key_pair_file.write(key_pair)
key_pair_file.close()

Create a role for the EC2 instance we are going to use. Read for detailed information about [IAM roles for Amazon EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html).

Follow steps here to [create an IAM role](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html#create-iam-role). Note down the role name and role ARN; the role name will be used when we launch the EC2 instance, and role ARN will be needed to create inline policy.

After creation, make sure the following policies are attached to role:

- AmazonS3FullAccess 
- AmazonSSMManagedInstanceCore 
- CloudWatchAgentAdminPolicy 


Locate the same SageMaker role used for running this notebook in [Demo Setup](#Demo-Setup) in [IAM console](https://console.aws.amazon.com/iam), click `Add inline policy` button on the role summary page, choose JSON format and replace the content with below statement:

Before copying the following content, make sure you use the EC2 role ARN you just created in the `Resource` field for `iam:PassRole` action.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::<account>:role/<role-name>"
        }
    ]
}
```

Launch an EC2 C5 instance. In this example we will use an AWS deep learning AMI.

In [ ]:
ami_map = {
    "us-east-1": "ami-063585f0e06d22308",
    "us-east-2": "ami-01bd6a1621a6968d7",
    "us-west-2": "ami-0bc87a16c757a7f07",
    "eu-central-1": "ami-01227276a4e5a4a31",
    "ap-northeast-1": "ami-03b8cfea5460e4881",
    "eu-west-1": "ami-006ff58f5247c50eb",
}

In [ ]:
ec2_profile_name = "<ec2_role_name>"  # the name of the role created for EC2

ec2_instance = ec2_client.run_instances(
    ImageId=ami_map[region],
    MinCount=1,
    MaxCount=1,
    InstanceType="c5.large",
    KeyName=ec2_key_name,
    IamInstanceProfile={
        "Name": ec2_profile_name,
    },
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "edge-manager-notebook"}],
        }
    ],
)

In [ ]:
instance_id = ec2_instance["Instances"][0]["InstanceId"]
print(instance_id)

## Compile Model using SageMaker Neo

Create a SageMaker client.

In [ ]:
sagemaker_client = boto3.client("sagemaker", region_name=region)

### Download pretrained Darknet model

In [ ]:
!wget -O yolov3-tiny.cfg https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg?raw=true
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

In [ ]:
import tarfile

with tarfile.open("yolov3-tiny.tar.gz", mode="w:gz") as archive:
    archive.add("yolov3-tiny.cfg")
    archive.add("yolov3-tiny.weights")

In [ ]:
darknet_model_path = sess.upload_data("yolov3-tiny.tar.gz", bucket, folder)

**Note**: When calling ``create_compilation_job()`` the user is expected to provide all the correct input shapes required by the model for successful compilation. If using a different model, you will need to specify the framework and data shape correctly.

In [ ]:
darknet_model_data_shape = '{"data":[1,3,416,416]}'
darknet_model_framework = "darknet"
target_os = "LINUX"
target_arch = "X86_64"

In [ ]:
import time

darknet_compilation_job_name = "Sagemaker-Edge-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % darknet_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=darknet_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": darknet_model_path,
        "DataInputConfig": darknet_model_data_shape,
        "Framework": darknet_model_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetPlatform": {"Arch": target_arch, "Os": target_os},
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=darknet_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        print(response)
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

## Package Model using Sagemaker Edge Manager

In this section, we will walk through packaging two models that achieve different goals. One is a Keras based Image Classification model and the other is a Darknet based Object Detection Model. 

### Package Darknet Model

Before we can deploy the compiled model to edge devices, we need to package the model using SageMaker Edge Manager cloud service.

In [ ]:
darknet_packaged_model_name = "darknet-model"
darknet_model_version = "1.0"
darknet_model_package = "{}-{}.tar.gz".format(darknet_packaged_model_name, darknet_model_version)

In [ ]:
darknet_packaging_job_name = darknet_compilation_job_name + "-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
    },
    ModelName=darknet_packaged_model_name,
    ModelVersion=darknet_model_version,
    EdgePackagingJobName=darknet_packaging_job_name,
    CompilationJobName=darknet_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=darknet_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        print(job_status)
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
print("Done!")

In [ ]:
darknet_model_data = job_status["ModelArtifact"]

### Download pretrained Keras model

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save("mobilenet_v2.h5")

In [ ]:
import tarfile

with tarfile.open("mobilenet_v2.tar.gz", mode="w:gz") as archive:
    archive.add("mobilenet_v2.h5")

In [ ]:
keras_model_path = sess.upload_data("mobilenet_v2.tar.gz", bucket, folder)

**Note**: When calling ``create_compilation_job()`` the user is expected to provide all the correct input shapes required by the model for successful compilation. If using a different model, you will need to specify the framework and data shape correctly.

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = "keras"
target_os = "LINUX"
target_arch = "X86_64"

In [ ]:
import time

keras_compilation_job_name = "Sagemaker-Edge-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=keras_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": keras_model_path,
        "DataInputConfig": keras_model_data_shape,
        "Framework": keras_model_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetPlatform": {"Arch": target_arch, "Os": target_os},
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=keras_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        print(response)
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

### Package Keras Model

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0"
keras_model_package = "{}-{}.tar.gz".format(keras_packaged_model_name, keras_model_version)

In [ ]:
keras_packaging_job_name = keras_compilation_job_name + "-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=keras_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        print(job_status)
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
print("Done!")

In [ ]:
keras_model_data = job_status["ModelArtifact"]

### Create AWS IoT thing

SageMaker Edge Manager uses AWS IoT Core to authenticate the device so we can make calls to SageMaker Edge Manager endpoints in AWS Cloud. 

In order for an edge device to use AWS services, it is necessary for it to first authenticate. We recommend doing this via AWS IoT based authentication, for more details refer [here](https://docs.aws.amazon.com/iot/latest/developerguide/authorizing-direct-aws.html) and [here](https://aws.amazon.com/blogs/security/how-to-eliminate-the-need-for-hardcoded-aws-credentials-in-devices-by-using-the-aws-iot-credentials-provider/).

In [ ]:
iot_client = boto3.client("iot", region_name=region)

In [ ]:
iot_thing_name = "sagemaker-edge-thing-demo"
iot_thing_type = "SagemakerEdgeDemo"

In [ ]:
iot_client.create_thing_type(thingTypeName=iot_thing_type)

In [ ]:
iot_client.create_thing(thingName=iot_thing_name, thingTypeName=iot_thing_type)

### Create Device Fleet

#### Create IAM role for device fleet

Configure an IAM role in your AWS account that will be assumed by the credentials provider on behalf of the devices in your device fleet. 

**Notice**: The name of the role must start with `SageMaker`.

Go to [IAM console](https://console.aws.amazon.com/iam), create role for IoT, attach the following policies:

- AmazonSageMakerEdgeDeviceFleetPolicy

Add the statement to trust relationship:
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
```

Note down the role ARN, it will be later used for creating the device fleet.

In [ ]:
device_fleet_name = "demo-device-fleet" + str(time.time()).split(".")[0]

# This is the role you created following the instructions in the above cell
device_fleet_role_arn = "<arn:aws:iam::<account>:role/SageMaker*>"

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=device_fleet_role_arn,
    OutputConfig={"S3OutputLocation": s3_compilation_output_location},
)

#### Register device to the fleet

In [ ]:
device_name = (
    "sagemaker-edge-demo-device" + str(time.time()).split(".")[0]
)  # device name should be 36 charactors

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

### Create and register client certificate with AWS IoT

Create private key, public key, and X.509 certificate files to register and activate the certificate with AWS IoT. 

In [ ]:
iot_cert = iot_client.create_keys_and_certificate(setAsActive=True)

Save the files and upload to S3 bucket, these files will be used to provide credentials on device to communicate with AWS services.

In [ ]:
with open("./iot.pem.crt", "w") as f:
    for line in iot_cert["certificatePem"].split("\n"):
        f.write(line)
        f.write("\n")

In [ ]:
with open("./iot_key.pem.key", "w") as f:
    for line in iot_cert["keyPair"]["PrivateKey"].split("\n"):
        f.write(line)
        f.write("\n")

In [ ]:
with open("./iot_key_pair.pem.key", "w") as f:
    for line in iot_cert["keyPair"]["PublicKey"].split("\n"):
        f.write(line)
        f.write("\n")

Associate the role alias generated from `create_device_fleet()` with AWS IoT.

In [ ]:
role_alias_name = "SageMakerEdge-" + device_fleet_name

role_alias = iot_client.describe_role_alias(roleAlias=role_alias_name)

We created and registered a certificate with AWS IoT earlier for successful authentication of your device. Now, we need to create and attach a policy to the certificate to authorize the request for the security token.

In [ ]:
alias_policy = {
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": "iot:AssumeRoleWithCertificate",
        "Resource": role_alias["roleAliasDescription"]["roleAliasArn"],
    },
}

In [ ]:
policy_name = "aliaspolicy-" + str(time.time()).split(".")[0]
aliaspolicy = iot_client.create_policy(
    policyName=policy_name,
    policyDocument=json.dumps(alias_policy),
)

In [ ]:
iot_client.attach_policy(policyName=policy_name, target=iot_cert["certificateArn"])

Obtain your AWS account-specific endpoint for the credentials provider.

In [ ]:
iot_endpoint = iot_client.describe_endpoint(endpointType="iot:CredentialProvider")

In [ ]:
endpoint = "https://{}/role-aliases/{}/credentials".format(
    iot_endpoint["endpointAddress"], role_alias_name
)

Get offical Amazon Root CA file and upload to S3 bucket. 

In [ ]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

Use the endpoint to make an HTTPS request to the credentials provider to return a security token. The following example command uses curl, but you can use any HTTP client.

**Optional: verify the credentials.**


In [ ]:
!curl --cert iot.pem.crt --key iot_key.pem.key --cacert AmazonRootCA1.pem $endpoint

If the certificate can be verified with the endpoint without error, upload certificate files to S3 bucket.

These files will be used in the [Setup Sagemaker Edge Manager Agent](#Setup-Sagemaker-Edge-Manager-Agent) section on EC2/device as Credential Provider.

In [ ]:
root_ca_path = sess.upload_data("AmazonRootCA1.pem", bucket, iot_folder)
device_cert_path = sess.upload_data("iot.pem.crt", bucket, iot_folder)
device_key_path = sess.upload_data("iot_key.pem.key", bucket, iot_folder)

## Inference on the Edge

In our example, we will use [AWS Systems Manager](https://docs.aws.amazon.com/systems-manager/latest/userguide/ssm-agent.html) to remotely perform actions on the EC2 instance. To see the SSM logs from CloudWatch, refer to [Install CouldWatch Agent section](#(Optional)Install-CloudWatch-Agent). 

Execution status of send_command is available in [AWS Systems Manager console](https://console.aws.amazon.com/systems-manager/run-command/complete-commands) command history.

In [ ]:
ssm_client = boto3.client("ssm", region_name=region)

### Setup SageMaker Edge Agent

Download SageMaker Edge Agent binary examples to EC2 instance.

Please fetch the latest version of binaries from SageMaker Edge release bucket. For more information about [Inference engine (Edge Manager agent)](https://docs.aws.amazon.com/sagemaker/latest/dg/edge-device-fleet-about.html).

In [ ]:
release_bucket_map = {
    "armv8": "sagemaker-edge-release-store-us-west-2-linux-armv8",
    "linux": "sagemaker-edge-release-store-us-west-2-linux-x64",
    "win64": "sagemaker-edge-release-store-us-west-2-windows-x64",
    "win32": "sagemaker-edge-release-store-us-west-2-windows-x86",
}

In [ ]:
# In this example, we will run inference on Linux
release_bucket = release_bucket_map["linux"]

To download the artifacts, specify the `VERSION`. The `VERSION` is broken into three components: `<MAJOR_VERSION>.<YYYY-MM-DD>-<SHA-7>`, where:

- MAJOR_VERSION: The release version. The release version is currently set to 1.

- `<YYYY-MM-DD>`: Time stamp of the artifacts release.

- SHA-7: repository commit ID the release is built from.

We suggest you use the latest artifact release time stamp. Use the following to get the latest time stamp.

In [ ]:
!aws s3 ls s3://$release_bucket/Releases/ | sort -r

In [ ]:
# A version string from the above cell
version = "<version>"

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "mkdir /demo",
            "aws s3 cp s3://{}/Releases/{}/{}.tgz demo.tgz".format(
                release_bucket, version, version
            ),
            "tar -xf demo.tgz -C /demo",
            "cd /demo/bin",
            "chmod +x sagemaker_edge_agent_binary",
            "chmod +x sagemaker_edge_agent_client_example",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

Get model signing root certificates from the release bucket.

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "cd /demo",
            "mkdir certificates",
            "aws s3 cp s3://{}/Certificates/{}/{}.pem certificates".format(
                release_bucket, region, region
            ),
            "chmod 400 certificates/*",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

Download IoT certificates, private key, models, and test images to the EC2 instance.

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "cd /demo",
            "mkdir iot-credentials",
            "cd iot-credentials",
            "aws s3 cp " + root_ca_path + " .",
            "aws s3 cp " + device_cert_path + " .",
            "aws s3 cp " + device_key_path + " .",
            "cd /demo",
            "aws s3 cp " + darknet_img_path + " .",
            "aws s3 cp " + darknet_model_data + " .",
            "mkdir darknet_model",
            "tar -xf " + darknet_model_package + " -C darknet_model",
            "aws s3 cp " + keras_img_path + " .",
            "aws s3 cp " + keras_model_data + " .",
            "mkdir keras_model",
            "tar -xf " + keras_model_package + " -C keras_model",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Configure SageMaker Edge Agent

Generate SageMaker Edge Agent configuration file. [For more information see the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/edge-device-fleet-about.html#edge-device-fleet-running-agent).

In [ ]:
sagemaker_edge_config = {
    "sagemaker_edge_core_device_name": device_name,
    "sagemaker_edge_core_device_fleet_name": device_fleet_name,
    "sagemaker_edge_core_capture_data_buffer_size": 30,
    "sagemaker_edge_core_capture_data_batch_size": 10,
    "sagemaker_edge_core_capture_data_push_period_seconds": 4,
    "sagemaker_edge_core_folder_prefix": "demo_capture",
    "sagemaker_edge_core_region": region,
    "sagemaker_edge_core_root_certs_path": "/demo/certificates",
    "sagemaker_edge_provider_aws_ca_cert_file": "/demo/iot-credentials/AmazonRootCA1.pem",
    "sagemaker_edge_provider_aws_cert_file": "/demo/iot-credentials/iot.pem.crt",
    "sagemaker_edge_provider_aws_cert_pk_file": "/demo/iot-credentials/iot_key.pem.key",
    "sagemaker_edge_provider_aws_iot_cred_endpoint": endpoint,
    "sagemaker_edge_provider_provider": "Aws",
    "sagemaker_edge_provider_s3_bucket_name": bucket,
    "sagemaker_edge_core_capture_data_destination": "Cloud",
}

In [ ]:
edge_config_file = open("sagemaker_edge_config.json", "w")
json.dump(sagemaker_edge_config, edge_config_file, indent=6)
edge_config_file.close()

Upload SageMaker Edge Agent configuration to an S3 bucket.

In [ ]:
config_path = sess.upload_data("sagemaker_edge_config.json", bucket, iot_folder)

Download the config file to the EC2 instance.

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={"commands": ["#!/bin/bash", "aws s3 cp " + config_path + " /demo"]},
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Launch SageMaker Edge Agent

Initialize the SageMaker Edge Manager agent. Note that we are using a long timeout on this command. In production, we recommend running the agent as a daemon, using a service like systemd.

In [ ]:
agent_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    TimeoutSeconds=24 * 60 * 60,
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "rm -f /tmp/sagemaker_edge_agent_example.sock",
            "./bin/sagemaker_edge_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c sagemaker_edge_config.json",
        ]
    },
)

### Load Model

In this section, we show the model management capabilities offered by SageMaker Edge Manager. We will load the two compiled and packaged models using the Agent. This keeps both models ready to run inference with. As you will see, once the models are loaded you can run multiple inferences as many times as necessary until the models are unloaded. This relieves the client applications from the logic and operational burden of managing them separately. These models are now simply an API away from running inference with.

When loading the model with the SageMaker Edge Agent, the argument to the API points the Agent to a directory containing the packaged model (without any extraneous files within the directory). 

#### Load darknet model

`darknet_model` is the path containing the packaged model in this notebook. `demo-darknet` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_darknet_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel darknet_model demo-darknet",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_darknet_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Load keras model

`keras_model` is the path containing the packaged model in this notebook. `demo-keras` is the name given to this model. This name will be used later to refer to this model for, making predictions, capturing data, unload.

In [ ]:
load_keras_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example LoadModel keras_model demo-keras",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=load_keras_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### List Models

This API simply lists all the models and their names that are loaded by the Agent. Note that the names shown here are same as the ones provided during the LoadModel in the previous sections.

In [ ]:
list_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={"commands": ["cd /demo", "./bin/sagemaker_edge_agent_client_example ListModels"]},
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=list_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Run Predict

In this API, we pass the model name, input data file that will be directly fed into the neural network, input tensor name that was passed earlier during the compilation phase, along with it's size and shape.

#### Run prediction on darknet model

In [ ]:
darknet_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-darknet darknet.bmp data 416 416 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_predict_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Run prediction on keras model

In [ ]:
keras_predict_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example Predict demo-keras keras.bmp input_1 224 224 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_predict_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Capture Data

Capture the inputs and outputs of an inference call to cloud or disk. The specific parameters were configured earlier in the config file. 

In [ ]:
darknet_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-darknet darknet.bmp data 416 416 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=darknet_capture_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

In [ ]:
keras_capture_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example PredictAndCapture demo-keras keras.bmp input_1 224 224 3",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=keras_capture_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Unload Model

After unloading a model, the same name can be reused for future `LoadModel` APIs calls.

In [ ]:
unload_model_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "cd /demo",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-darknet",
            "./bin/sagemaker_edge_agent_client_example UnloadModel demo-keras",
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=unload_model_out["Command"]["CommandId"],
    InstanceId=instance_id,
)

## Clean Up

Stop the Agent

In [ ]:
ssm_client.cancel_command(CommandId=agent_out["Command"]["CommandId"], InstanceIds=[instance_id])

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(InstanceIds=[instance_id])

Detach and delete policy

In [ ]:
iot_client.detach_policy(policyName=policy_name, target=iot_cert["certificateArn"])

iot_client.delete_policy(policyName=policy_name)

Deregister device and delete device fleet

In [ ]:
sagemaker_client.deregister_devices(DeviceFleetName=device_fleet_name, DeviceNames=[device_name])

sagemaker_client.delete_device_fleet(DeviceFleetName=device_fleet_name)

## Appendix

### (Optional)Install CloudWatch Agent 

In [ ]:
CW_log_config = {
    "agent": {
        "metrics_collection_interval": 10,
        "logfile": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
    },
    "logs": {
        "logs_collected": {
            "files": {
                "collect_list": [
                    {
                        "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/amazon-cloudwatch-agent.log",
                        "log_group_name": "amazon-cloudwatch-agent.log",
                        "log_stream_name": "amazon-cloudwatch-agent.log",
                        "timezone": "UTC",
                    },
                    {
                        "file_path": "/opt/aws/amazon-cloudwatch-agent/logs/test.log",
                        "log_group_name": "test.log",
                        "log_stream_name": "test.log",
                        "timezone": "Local",
                    },
                ]
            }
        },
        "log_stream_name": "my_log_stream_name",
        "force_flush_interval": 15,
    },
}

In [ ]:
CW_file = open("cloudwatch.json", "w")
json.dump(CW_log_config, CW_file, indent=6)
CW_file.close()

In [ ]:
CW_config_path = sess.upload_data("cloudwatch.json", bucket, iot_folder)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "aws s3 cp "
            + CW_config_path
            + " /opt/aws/amazon-cloudwatch-agent/etc/amazon-cloudwatch-agent.json",
        ]
    },
)

In [ ]:
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "wget https://s3.amazonaws.com/amazoncloudwatch-agent/ubuntu/amd64/latest/amazon-cloudwatch-agent.deb",
            "sudo dpkg -i -E ./amazon-cloudwatch-agent.deb",
        ]
    },
)

Install Cloud Watch Agent to SSM agent.

In [ ]:
ssm_client.send_command(
    DocumentName="AWS-ConfigureAWSPackage",
    DocumentVersion="1",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Targets=[
        {"Key": "InstanceIds", "Values": [instance_id]},
    ],
    TimeoutSeconds=600,
    Parameters={"action": ["Install"], "name": ["AmazonCloudWatchAgent"]},
    MaxConcurrency="50",
    MaxErrors="0",
)

To debug with CloudWatch, add a parameter `CloudWatchOutputConfig` to `send_command`
```
CloudWatchOutputConfig={
    'CloudWatchOutputEnabled': True
}
```

Example:
```
ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    CloudWatchOutputConfig={
        'CloudWatchOutputEnabled': True
    },
    Parameters={
        'commands':[
            "cd /demo",
            "./bin/neo_agent_binary -a /tmp/sagemaker_edge_agent_example.sock -c neo_config.json" 
        ]
    }
)
```

Running log can be found in cloud watch log group `/aws/ssm/AWS-RunShellScript`